In [1]:
import pandas as pd
import numpy as np

In [2]:
df = pd.read_csv("data/train.csv")

In [3]:
df = df.replace(r'^\s*$', np.nan, regex=True)
#Filling with mode for the other columns
df['STATE'].fillna(df['STATE'].mode()[0], inplace=True)
df['OCCUPATION'].fillna('U1', inplace=True)
df['INCOME_GROUP'].fillna(df['INCOME_GROUP'].mode()[0], inplace=True)
df['LOYALTY_PROGRAM'].fillna('U2', inplace=True)

In [4]:
df

ID           STATE  AGE     OCCUPATION INCOME_GROUP  \
0            1   Uttar Pradesh   28             U1          Low   
1            2  Madhya Pradesh   27       Salaried          Low   
2            3     West Bengal   62             U1         High   
3            4      Tamil Nadu   27       Salaried          Low   
4            5   Uttar Pradesh   40  Self employed         High   
...        ...             ...  ...            ...          ...   
149995  149996           Assam   29       Salaried          Low   
149996  149997  Andhra Pradesh   53  Self employed       Medium   
149997  149998          Odisha   31             U1          Low   
149998  149999  Andhra Pradesh   54  Self employed         High   
149999  150000     Maharashtra   60             U1       Medium   

        CUSTOMER_SINCE LOYALTY_PROGRAM  PAST_PURCHASE PURCHASE  
0                 2018              No           4937       No  
1                 2018              No          10834       No  
2                 2013              U2           8610      Yes  
3                 2018              No          14374      Yes  
4                 2015              No          13158       No  
...                ...             ...            ...      ...  
149995            2017              No           6241       No  
149996            2010              No          29447       No  
149997            2018              No           4856       No  
149998            2010             Yes          10031      Yes  
149999            2013              U2           7761      Yes  

[150000 rows x 9 columns]

In [5]:
#Dropping the ID column 
df = df.drop(['ID'], axis = 1)

In [6]:
df['LOYALTY_PROGRAM'] = df['LOYALTY_PROGRAM'].map({'Yes': 1, 'No': 0})
df['PURCHASE'] = df['PURCHASE'].map({'Yes': 1, 'No': 0})

In [7]:
from sklearn.model_selection import train_test_split

In [8]:
#Split the dataset into 70(train) - 30(test)
X = df.drop(['PURCHASE'], axis=1)
y = df['PURCHASE']

In [9]:
import category_encoders as ce

In [10]:
# Define catboost encoder
cbe_encoder = ce.cat_boost.CatBoostEncoder()
  
# Fit encoder and transform the features
cbe_encoder.fit(X, y)
train_cbe = cbe_encoder.transform(X)

In [11]:
X_train, X_test, y_train, y_test = train_test_split(train_cbe, y, test_size = 0.3, random_state = 0)

In [12]:
from sklearn.metrics import accuracy_score
from sklearn.metrics import f1_score
from xgboost import XGBClassifier
# fit model no training data
model = XGBClassifier()
model.fit(X_train, y_train)
y_pred = model.predict(X_test)
predictions = [round(value) for value in y_pred]
# evaluate predictions
accuracy = accuracy_score(y_test, predictions)
print("Accuracy: %.2f%%" % (accuracy * 100.0))
print("F1 Score: ",f1_score(y_test, predictions))

C:\Users\vishn\anaconda3\lib\site-packages\xgboost\sklearn.py:1146: UserWarning: The use of label encoder in XGBClassifier is deprecated and will be removed in a future release. To remove this warning, do the following: 1) Pass option use_label_encoder=False when constructing XGBClassifier object; and 2) Encode your labels (y) as integers starting with 0, i.e. 0, 1, 2, ..., [num_class - 1].
  warnings.warn(label_encoder_deprecation_msg, UserWarning)


[09:08:08] WARNING: C:/Users/Administrator/workspace/xgboost-win64_release_1.4.0/src/learner.cc:1095: Starting in XGBoost 1.3.0, the default evaluation metric used with the objective 'binary:logistic' was changed from 'error' to 'logloss'. Explicitly set eval_metric if you'd like to restore the old behavior.
Accuracy: 85.74%
F1 Score:  0.6361213495888858


In [13]:
from catboost import CatBoostClassifier
# fit model no training data
model = CatBoostClassifier()
model.fit(X_train, y_train)
y_pred = model.predict(X_test)
predictions = [round(value) for value in y_pred]
# evaluate predictions
accuracy = accuracy_score(y_test, predictions)
print("Accuracy: %.2f%%" % (accuracy * 100.0))
print("F1 Score: ",f1_score(y_test, predictions))

Learning rate set to 0.075159
0:	learn: 0.6321978	total: 181ms	remaining: 3m
1:	learn: 0.5821443	total: 204ms	remaining: 1m 41s
2:	learn: 0.5413724	total: 227ms	remaining: 1m 15s
3:	learn: 0.5082903	total: 246ms	remaining: 1m 1s
4:	learn: 0.4810570	total: 266ms	remaining: 53s
5:	learn: 0.4597338	total: 285ms	remaining: 47.2s
6:	learn: 0.4419563	total: 305ms	remaining: 43.3s
7:	learn: 0.4278963	total: 325ms	remaining: 40.2s
8:	learn: 0.4146809	total: 346ms	remaining: 38.1s
9:	learn: 0.4053181	total: 362ms	remaining: 35.8s
10:	learn: 0.3970745	total: 384ms	remaining: 34.5s
11:	learn: 0.3898942	total: 412ms	remaining: 33.9s
12:	learn: 0.3844795	total: 430ms	remaining: 32.6s
13:	learn: 0.3796952	total: 452ms	remaining: 31.8s
14:	learn: 0.3757988	total: 476ms	remaining: 31.2s
15:	learn: 0.3722780	total: 500ms	remaining: 30.7s
16:	learn: 0.3703314	total: 521ms	remaining: 30.1s
17:	learn: 0.3677233	total: 548ms	remaining: 29.9s
18:	learn: 0.3655449	total: 569ms	remaining: 29.4s
19:	learn: 0.3

In [14]:
print("Accuracy: %.2f%%" % (accuracy * 100.0))
print("F1 Score: ",f1_score(y_test, predictions))

Accuracy: 85.82%
F1 Score:  0.638981612446959


In [17]:
from lightgbm import LGBMClassifier
# fit model no training data
model = LGBMClassifier()
model.fit(X_train, y_train)
y_pred = model.predict(X_test)
predictions = [round(value) for value in y_pred]
# evaluate predictions
accuracy = accuracy_score(y_test, predictions)
print("Accuracy: %.2f%%" % (accuracy * 100.0))
print("F1 Score: ",f1_score(y_test, predictions))

Accuracy: 85.90%
F1 Score:  0.6319939699657912


As you can see, we get much better results when we perfom the classification by considering missing values as a seperate category. We an F1 score of around 0.63 which is almost a 2 fold increase from when compared to the last time. 